In [72]:
import pandas as pd
import requests
import json
import datetime

In [73]:
gare = "Nantes"

# Max 250
nb_trains = 20

In [74]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

In [75]:
dic_gare = {"Paris - Montparnasse - Hall 1 & 2": "SNCF:87391003",
            "Paris - Gare de Lyon - Hall 1 & 2": "SNCF:87686006",
            "Paris Est": "SNCF:87113001",
            "Paris Nord": "SNCF:87271007",
            "Lyon Part Dieu": "SNCF:87723197",
            "Rennes": "SNCF:87471003",
            "Bordeaux Saint-Jean": "SNCF:87581009",
            "Nantes": "SNCF:87481002",
            "Montpellier Saint-Roch": "SNCF:87773002",
            "Nîmes Centre": "SNCF:87775007",
            "Strasbourg": "SNCF:87212027",
            "Marseille - Saint-Charles": "SNCF:87751008",
            "Lille Flandres": "SNCF:87286005",
            "Toulouse Matabiau": "SNCF:87611004",
            "Alès": "SNCF:87775288"
            }

In [87]:
link = 'https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:' + dic_gare[gare] + '/arrivals?count=' + str(nb_trains)
req = requests.get(link,auth=(token, ''))
#print(link)

In [77]:
doc = json.loads(req.text)
row = len(doc['arrivals'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 20


In [78]:
df = pd.DataFrame(doc['arrivals'])
df_gare = pd.DataFrame(list(df['display_informations']))
df_heure = pd.DataFrame(list(df['stop_date_time']))
df_id = pd.DataFrame(list(df['links']))
df_id = pd.DataFrame(list(df_id[1]))

In [79]:
def del_day(string):
    return string[9:]

df_heure['arrival_date_time'] = df_heure['arrival_date_time'].apply(del_day)
df_heure['base_arrival_date_time'] = df_heure['base_arrival_date_time'].apply(del_day)

In [80]:
def conv_min(string):
    return int(string[0:2])*60 + int(string[2:4])

df_heure['retard'] = df_heure['arrival_date_time'].apply(conv_min) - df_heure['base_arrival_date_time'].apply(conv_min)

In [81]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_heure['heure'] = df_heure['arrival_date_time'].apply(str_tps)
df_heure['old_heure'] = df_heure['base_arrival_date_time'].apply(str_tps)

In [82]:
def del_par(string):
    index = string.find("(")
    return string[:index]

df_gare['direction'] = df_gare['direction'].apply(del_par)

In [83]:
df_gare = df_gare[['direction','network','trip_short_name']]
df_gare.rename(columns = {'direction':'Destination'}, inplace = True)
df_gare.rename(columns = {'network':'Train'}, inplace = True)
df_gare.rename(columns = {'trip_short_name':'Numéro'}, inplace = True)

df_gare['Arrivée (réelle)'] = df_heure['heure']
df_gare['Arrivée (prévue)'] = df_heure['old_heure']
df_gare['Retard (min)'] = df_heure['retard']
df_gare['id'] = df_id['id']

In [84]:
def get_name(string):
    string = string[10:]
    index_fin = string.find("', 'links'")
    return string[:index_fin]

arrets = []
causes = []

for i in df_gare.index:
    id = df_gare.iloc[i]['id']
    
    if 'RealTime' in id:
        index_id = id.index("RealTime")
        id = id[:index_id-1]

    link_voyage = 'https://api.sncf.com/v1/coverage/sncf/vehicle_journeys/' + id
    req_arret = requests.get(link_voyage ,auth=(token, ''))
    doc_voyage = json.loads(req_arret.text)

    df_arret = pd.DataFrame(doc_voyage['vehicle_journeys'])
    df_arret = pd.DataFrame(list(df_arret['stop_times']))
    df_arret = df_arret.T
    df_arret = pd.DataFrame(list(df_arret[0]))
    df_arret['stop_point'] = df_arret['stop_point'].astype('str').apply(get_name)

    if df_gare.iloc[i]['Retard (min)'] != 0:
        df_retard = pd.DataFrame(doc_voyage['disruptions'])
        if 'messages' in df_retard:
            df_retard = pd.DataFrame(list(df_retard['messages'][0]))
            causes.append(df_retard.iloc[0]['text'])
        else:
            causes.append("Retard non expliqué")
    else:
        causes.append("")

    liste_arrets = list(df_arret['stop_point'])

    arrets.append(liste_arrets)

In [85]:
df_gare['Cause'] = causes
df_gare['Arrêts'] = arrets
df_gare = df_gare.drop(['id'], axis=1)

In [86]:
df_gare

,Destination,Train,Numéro,Arrivée (réelle),Arrivée (prévue),Retard (min),Cause,Arrêts
0,Nantes,TER,859656,15:37:00,15:37:00,0,,"[Cholet, Torfou-Tiffauges, Boussay - La Bruffi..."
1,Nantes,TER,859421,15:44:00,15:34:00,10,Régulation du trafic,"[Ancenis, Oudon, Le Cellier, Mauves-sur-Loire,..."
2,Nantes,TER,859118,15:44:00,15:44:00,0,,"[Saint-Gilles-Croix-de-Vie, Saint-Hilaire-de-R..."
3,Nantes,TER,858070,15:48:00,15:48:00,0,,"[Le Croisic, Batz-sur-Mer, Le Pouliguen, La Ba..."
4,Nantes,TGV INOUI,8875,15:54:00,15:54:00,0,,"[Paris - Montparnasse - Hall 1 & 2, Le Mans, S..."
5,Nantes,TER,858868,15:54:00,15:54:00,0,,"[Les Sables-d'Olonne"", 'links': [], 'coord': {..."
6,Nantes,Intercités,4502,16:04:00,16:04:00,0,,"[Lyon Perrache, Lyon Part Dieu, Roanne, Saint-..."
7,Nantes,TER,859780,16:20:00,16:20:00,0,,"[Clisson, Gorges, Le Pallet, La Haie-Fouassièr..."
8,Nantes,TER,858020,16:21:00,16:21:00,0,,"[Saint-Nazaire, Penhoët, La Croix de Méan, Mon..."
9,Nantes,TER,859509,16:29:00,16:29:00,0,,"[Sucé-sur-Erdre, La Chapelle - Aulnay, La Chap..."
